In [0]:
from pyspark.sql.functions import current_timestamp, lit

In [0]:
v_file_date = "2024-12-30"

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/commom_functions"

In [0]:
silver_folder_path

In [0]:
movies_df = spark.read.parquet(f"{silver_folder_path}/movies") \
                      .filter(f"file_date = '{v_file_date}'")

In [0]:
language_df = spark.read.parquet(f"{silver_folder_path}/languages")

In [0]:
movies_languages_df = spark.read.parquet(f"{silver_folder_path}/movies_languages") \
                                .filter(f"file_date = '{v_file_date}' ")

In [0]:
genres_df = spark.read.parquet(f"{silver_folder_path}/genres")

In [0]:
movies_genres_df = spark.read.parquet(f"{silver_folder_path}/movies_genres") \
                    .filter(f"file_date = '{v_file_date}'")

In [0]:
languages_mov_lan_df = language_df.join(movies_languages_df, 
                                        language_df.language_id == movies_languages_df.language_id,
                                        "inner") \
                        .select(language_df.language_name,
                                movies_languages_df.movie_id)

In [0]:
genres_mov_gen_df = genres_df.join(movies_genres_df,
                                   genres_df.genre_id == movies_genres_df.genre_id,
                                   "inner") \
                    .select(genres_df.genre_name,
                            movies_genres_df.movie_id)

In [0]:
movie_filter_df = movies_df.filter("year_release_date >=2000")

In [0]:
results_movies_genres_languages_df = movie_filter_df.join(languages_mov_lan_df, 
                                                          movie_filter_df.movie_id == languages_mov_lan_df.movie_id,
                                                          "inner") \
                                                    .join(genres_mov_gen_df,
                                                          movie_filter_df.movie_id == genres_mov_gen_df.movie_id,
                                                          "inner")

In [0]:
results_df = results_movies_genres_languages_df \
            .select("title", "duration_time", "release_date", "vote_average", "language_name", "genre_name") \
            .withColumn("created_date", lit(v_file_date))

In [0]:
results_df.display()

In [0]:
results_order_by_df = results_df.orderBy(results_df.release_date.desc())

In [0]:
results_order_by_df.write.mode("overwrite").format("parquet").partitionBy('created_date').saveAsTable(f"movie_gold.results_movie_genre_language")

In [0]:
%sql
select *
from movie_gold.results_movie_genre_language;